# Extract Embeddings
This notebook is for building and testing a component for reading content from context, extracting their embeddings and putting the embeddings into the context.

In [ ]:
components_path = "../components/"

# register a magic command to save a cell to a file
from IPython.core.magic import register_cell_magic
import os

@register_cell_magic
def run_and_save(line, cell):
    # create directory if it does not exist
    if not os.path.exists(components_path):
        os.makedirs(components_path)

    'Run and save python code block to a file'
    path = components_path + line
    with open(path, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, path, 'exec')
    exec(code, globals())

In [ ]:
%%run_and_save extract_embeddings.py
from node_engine.libs.azure_openai_embeddings import AzureOpenAIEmbeddings
from node_engine.models.node_engine_component import NodeEngineComponent


class ExtractEmbeddings(NodeEngineComponent):
    description = "Extract embeddings from text. Pass the text in the context and the embeddings will be returned in the context."

    default_config = {"service": "AZUREOPENAI_EMBEDDINGS"}

    reads_from = {
        "context": "as defined in config",
        "config": {
            "source": {
                "type": "string",
                "description": "The key in the context containing the text to be embedded.",
                "required": True,
            },
            "target": {
                "type": "string",
                "description": "The key in the context to store the embeddings.",
                "required": True,
            },
        },
    }

    writes_to = {
        "context": "as defined in config",
    }

    sample_input = {
        "key": "sample",
        "session_id": "123456",
        "context": {
            "memories": [
                "Offering help: Assistant offered help and encouraged the user to ask questions if needed."
            ],
        },
        "flow": [
            {
                "key": "start",
                "name": "ExtractEmbeddings",
                "config": {
                    "source": "memories",
                    "target": "memories_embeddings",
                },
            },
        ],
    }

    sample_output = {
        "key": "sample",
        "session_id": "123456",
        "context": {
            "memories": [
                "Offering help: Assistant offered help and encouraged the user to ask questions if needed."
            ],
            "memories_embeddings": [
                {
                    "item": "Offering help: Assistant offered help and encouraged the user to ask questions if needed.",
                    "embedding": [
                        -0.0003509521484375,
                        0.00018310546875,
                        0.0001220703125,
                    ],
                }
            ],
        },
    }

    azure_openai_embeddings = AzureOpenAIEmbeddings()

    async def execute(self):
        # extract embeddings

        self.log("Extract embeddings started")

        # get source from context
        source_key = self.config.get("source")
        if not source_key:
            return self.exit_with_error("Source not found in config")
        source_data = self.context.get(source_key)
        if not source_data:
            return self.exit_with_error(
                "Source '{}' not found in context".format(source_key)
            )

        # get target from context
        target_key = self.config.get("target")
        if not target_key:
            return self.exit_with_error("Target not found in config")

        service = self.config.get("service")

        # get embeddings
        embeddings = await self.azure_openai_embeddings.create(source_data, service)

        results = []
        for i in range(len(source_data)):
            results.append({"item": source_data[i], "embedding": embeddings[i]})

        # add embeddings to context
        self.context.set("target", results)

        self.log("Extract embeddings completed")

        return self.continue_flow()


In [ ]:
# Test it
from fastapi.encoders import jsonable_encoder
import json
print(json.dumps(jsonable_encoder(await ExtractEmbeddings.test()), indent=2))